This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Deep learning for text

## Natural-language processing: The bird's eye view

### Preparing text data:

1. Text standardization
2. Text splitting (tokenization)
3. Vocabulary indexing

All this can be done with explicit functions in python, but as with many other tasks, it is much quicker (and easier) using the TextVectorization layer in keras (later in the notebook).

The first part is also intented to understand the preprocessing of text.

# 1. Using explicit functions in python

In [ ]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)

    def tokenize(self, text):
        text = self.standardize(text)
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)



In [ ]:
vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)



In [ ]:
print(vectorizer.vocabulary)

{'': 0, '[UNK]': 1, 'i': 2, 'write': 3, 'erase': 4, 'rewrite': 5, 'again': 6, 'and': 7, 'then': 8, 'a': 9, 'poppy': 10, 'blooms': 11}


In [ ]:
print(vectorizer.inverse_vocabulary)

{0: '', 1: '[UNK]', 2: 'i', 3: 'write', 4: 'erase', 5: 'rewrite', 6: 'again', 7: 'and', 8: 'then', 9: 'a', 10: 'poppy', 11: 'blooms'}


In [ ]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [ ]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


# 2. Using keras tools

In [ ]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
    output_mode="int",
)

In [ ]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn,
)

In [ ]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset)

**Displaying the vocabulary**

In [ ]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [ ]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [ ]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


# 3. Two approaches for representing groups of words: Sets and sequences

### Preparing the IMDB movie reviews data

In [ ]:
# Descarga el dataset completo IMDB

!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  44.5M      0  0:00:01  0:00:01 --:--:-- 44.4M


In [ ]:
# Descomprime el .tar.gz descargado
# OJO SOLO FUNCIONA EN LINUX - Windows: hacer el untar desde sistema operativo

!tar -xf aclImdb_v1.tar.gz

In [ ]:
# Borra un directorio que no se va a utilizar

!rm -r aclImdb/train/unsup

In [ ]:
!cat ./aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [ ]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Displaying the shapes and dtypes of the first batch**

In [ ]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"Most people, when they think of expressionist cinema, look to the b&w German films of the silent and early sound eras--films that emphasized canted angles, extreme contrasts of light and dark, exaggerated performance, and occasional uses of surrealism to create a dreamlike atmosphere in order to diverge from traditional, naturalistic modes of cinematic representation. If we're willing to accept that the Germans were not the only filmmakers to create expressionist cinema (and that those above-mentioned characteristics are not prerequisites for expressionist film), then I would argue that Dodes'ka-den (DKD) is a prime example of this type of film. <br /><br />Like Dreams, DKD is a little unhinged for a Kurosawa film, dabbling, as it does, in the unreal. However, DKD is also, unlike Dreams, a great film and probably my favorite Kurosawa picture. Why? Mostly, I th

### Processing words as a set: The bag-of-words approach

#### Single words (unigrams) with binary encoding

**Preprocessing our datasets with a `TextVectorization` layer**

In [ ]:
text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**Inspecting the output of our binary unigram dataset**

In [ ]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


**Our model-building utility**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, 
              hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

**Training and testing the binary unigram model**

In [ ]:
model = get_model()
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)
]


In [ ]:
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)


Epoch 1/10
625/625 [==============================] - 17s 25ms/step - loss: 0.4145 - accuracy: 0.8270 - val_loss: 0.2870 - val_accuracy: 0.8860
Epoch 2/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2741 - accuracy: 0.9000 - val_loss: 0.2772 - val_accuracy: 0.8926
Epoch 3/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2414 - accuracy: 0.9151 - val_loss: 0.2788 - val_accuracy: 0.8960
Epoch 4/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2248 - accuracy: 0.9253 - val_loss: 0.2895 - val_accuracy: 0.8982
Epoch 5/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2182 - accuracy: 0.9300 - val_loss: 0.3002 - val_accuracy: 0.8978
Epoch 6/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2137 - accuracy: 0.9316 - val_loss: 0.3191 - val_accuracy: 0.8904
Epoch 7/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2105 - accuracy: 0.9337 - val_loss: 0.3206 - val_accuracy: 0.891

In [ ]:
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 11s 14ms/step - loss: 0.2940 - accuracy: 0.8873
Test acc: 0.887


#### Bigrams with binary encoding

**Configuring the `TextVectorization` layer to return bigrams**

In [ ]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

## what does Text Vectorization do

From
https://keras.io/getting_started/intro_to_keras_for_engineers/

https://keras.io/api/layers/preprocessing_layers/core_preprocessing_layers/text_vectorization/


In [ ]:
import numpy as np
# Example training data, of dtype `string`.
example_data = np.array([["This is the 1st sample."], 
                         ["And here's the 2nd sample."]])

# Create a TextVectorization layer instance. It can be configured to either
# return integer token indices, or a dense token representation (e.g. multi-hot
# or TF-IDF). The text standardization and text splitting algorithms are fully
# configurable.
example_vectorizer = TextVectorization(output_mode="multi_hot", 
                                       ngrams=2)

# Calling `adapt` on an array or dataset makes the layer generate a vocabulary
# index for the data, which can then be reused when seeing new data.
example_vectorizer.adapt(example_data)

# After calling adapt, the layer is able to encode any n-gram it has seen before
# in the `adapt()` data. Unknown n-grams are encoded via an "out-of-vocabulary"
# token.
integer_data = example_vectorizer(example_data)
print(integer_data)


tf.Tensor(
[[0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]], shape=(2, 17), dtype=float32)


**Training and testing the binary bigram model**

In [ ]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)



In [ ]:
model = get_model()
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only=True)
]


In [ ]:
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)


Epoch 1/10
625/625 [==============================] - 16s 24ms/step - loss: 0.3772 - accuracy: 0.8485 - val_loss: 0.2636 - val_accuracy: 0.8946
Epoch 2/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2429 - accuracy: 0.9135 - val_loss: 0.2655 - val_accuracy: 0.9000
Epoch 3/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2088 - accuracy: 0.9319 - val_loss: 0.2789 - val_accuracy: 0.8966
Epoch 4/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1883 - accuracy: 0.9421 - val_loss: 0.2961 - val_accuracy: 0.8984
Epoch 5/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1840 - accuracy: 0.9466 - val_loss: 0.3104 - val_accuracy: 0.9034
Epoch 6/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1818 - accuracy: 0.9471 - val_loss: 0.3165 - val_accuracy: 0.9008
Epoch 7/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1839 - accuracy: 0.9509 - val_loss: 0.3387 - val_accuracy: 0.896

In [ ]:
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 11s 14ms/step - loss: 0.2646 - accuracy: 0.8992
Test acc: 0.899


#### Bigrams with TF-IDF encoding

**Configuring the `TextVectorization` layer to return token counts**

In [ ]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count"
)

**Configuring `TextVectorization` to return TF-IDF-weighted outputs**

In [ ]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

## Training the TF-IDF bigram model

In [ ]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)



In [ ]:
model = get_model()
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True)
]


In [ ]:
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)


Epoch 1/10
625/625 [==============================] - 16s 25ms/step - loss: 0.4943 - accuracy: 0.7843 - val_loss: 0.2987 - val_accuracy: 0.8808
Epoch 2/10
625/625 [==============================] - 6s 9ms/step - loss: 0.3382 - accuracy: 0.8651 - val_loss: 0.2962 - val_accuracy: 0.8770
Epoch 3/10
625/625 [==============================] - 6s 10ms/step - loss: 0.3013 - accuracy: 0.8769 - val_loss: 0.3074 - val_accuracy: 0.8802
Epoch 4/10
625/625 [==============================] - 6s 9ms/step - loss: 0.2765 - accuracy: 0.8834 - val_loss: 0.3303 - val_accuracy: 0.8528
Epoch 5/10
625/625 [==============================] - 6s 9ms/step - loss: 0.2647 - accuracy: 0.8903 - val_loss: 0.3130 - val_accuracy: 0.8754
Epoch 6/10
625/625 [==============================] - 6s 9ms/step - loss: 0.2619 - accuracy: 0.8870 - val_loss: 0.3225 - val_accuracy: 0.8634
Epoch 7/10
625/625 [==============================] - 6s 9ms/step - loss: 0.2482 - accuracy: 0.8940 - val_loss: 0.3333 - val_accuracy: 0.8620
Epo

In [ ]:
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 11s 14ms/step - loss: 0.3127 - accuracy: 0.8645
Test acc: 0.865


## Testing the TF-IDF bigram model

In [ ]:
inputs = keras.Input(shape=(1,), 
                     dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

In [ ]:
#import tensorflow as tf
raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

96.47 percent positive
